In [1]:
import librosa
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from dataset import training_dataset
from preprocessing import perform_vad, resample_wav, padding
from torch.utils.data import DataLoader
import pickle

c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Feature extraction

## Training part

In [2]:
labels_path = 'train\\testing_list.txt'
with open(labels_path, "r") as f:
    lines = f.readlines()
file_paths = [line.strip() for line in lines]


['bed/0c40e715_nohash_0.wav',
 'bed/0ea0e2f4_nohash_0.wav',
 'bed/0ea0e2f4_nohash_1.wav',
 'bed/105a0eea_nohash_0.wav',
 'bed/1528225c_nohash_0.wav']

In [12]:
features_train = []

for i, file in tqdm(enumerate(file_paths), total=len(file_paths), leave=True):
    wav_file = os.path.join('train\\audio',file)
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    perform_vad(wav_file, wav_file2)
    padding(wav_file2, wav_file2, 1000)
    resample_wav(wav_file2, 8000)
    audio, sample_rate = librosa.load(wav_file2)
  
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    delta = librosa.feature.delta(mfccs)
    delta_delta = librosa.feature.delta(mfccs, order=2)
 
    obs = np.concatenate([mfccs, delta, delta_delta], axis=0)
    features_train.append([obs, label])

100%|██████████| 6835/6835 [06:57<00:00, 16.39it/s]


## Validation part

In [15]:
with open('features_training.pkl', 'wb') as f:
    pickle.dump(features_train, f)

In [2]:
labels_path = 'train\\validation_list.txt'
with open(labels_path, "r") as f:
    lines = f.readlines()
file_paths = [line.strip() for line in lines]

In [3]:
features_val = []

for i, file in tqdm(enumerate(file_paths), total=len(file_paths), leave=True):
    wav_file = os.path.join('train\\audio',file)
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    perform_vad(wav_file, wav_file2)
    padding(wav_file2, wav_file2, 1000)
    resample_wav(wav_file2, 8000)
    audio, sample_rate = librosa.load(wav_file2)
  
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    delta = librosa.feature.delta(mfccs)
    delta_delta = librosa.feature.delta(mfccs, order=2)
 
    obs = np.concatenate([mfccs, delta, delta_delta], axis=0)
    features_val.append([obs, label])

100%|██████████| 6798/6798 [07:23<00:00, 15.34it/s]


In [4]:
with open('features_validation.pkl', 'wb') as f:
    pickle.dump(features_val, f)